# 第2篇：DataFrame

## 第1部分：DataFrame简介

DataFrame是一个带有索引的二维数据结构，每列可以有自己的名字，并且可以有不同的数据类型，类似于 Excel 、SQL 表，或 Series 对象构成的字典。你可以把它想象成一个excel表格或者数据库中的一张表，DataFrame是最常用的Pandas对象。  
上一节我们引入了pandas的基本数据结构Series,它的基本属性包含index、value、name、dtype等，属于一维数据结构。DataFrame是pandas的第二大基本数据结构，可以看成由一个或多个Series构成的二维数据结构，其除了拥有和Series一样的index、value、name、dtype等属性外，最大的特点是还拥有column，即列或标签。从这个角度来讲，Series由于是一维数据结构，所有的数据可以看成有多行一列，可以直接通过行索引取到value，故其没有列名；而DataFrame属于二维数据结构，可以看成多行多列，直接通过行索引取到的是多个列对应的value，因而需要对每列标注名字来识别对应的value，因此有了column。
![](https://zhangyafei-1258643511.cos.ap-nanjing.myqcloud.com/Python/blog/dataframe1.png)
上图展示了DataFrame与Series之间的关系，我们可以发现二者的区别和联系：
- 区别：
> series，只是一个一维数据结构，它由index和value(最基本元素)组成。  
> dataframe，是一个二维结构，除了拥有index和value之外，还拥有column。
- 联系：
> dataframe由多个series组成，无论是行还是列，单独拆分出来都是一个series。

## 第2部分：DataFrame的创建

DataFrame 是最常用的 Pandas 对象，与 Series 一样，DataFrame 支持多种类型的输入数据：
- 方式1：字典套Series
- 方式2：字典套列表
- 方式3：列表套字典
- 方式4：列表套列表（二维数组）
- 方式5：DataFrame.from_dict
- 方式6：DataFrame.from_record
- 方式7：文件中读取

### 导入相关库

In [2244]:
import pandas as pd
import numpy as np

### 方式1：字典套Series
> data = {  
    "col1": pd.Series([],index),  
    "col2": pd.Series([],index),  
    "col3": pd.Series([],index)  
 } 

**这种方式有以下特点：**
- 字典的key为列名
- 字典的value为Series
- 若DataFrame没有指定index和columns，则以字典的key为列，Series的index为索引
- 若DataFrame指定了index或columns，会根据指定的columns去字典中筛选对应的key，根据指定的index去Series中筛选对应的索引，并将筛选的结果放入到生成的DataFrame中

In [2245]:
data = {
    "name": pd.Series(["Tom", "Bob", "Mary", "James", "Yafei"]),
    "age": pd.Series([18, 30, 25, 40, 22]),
    "city": pd.Series(["北京", "上海", "广州", "深圳", "晋城"])
}
user_info = pd.DataFrame(data)
user_info

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳
4,Yafei,22,晋城


**设置name为索引**

In [2246]:
# 指定索引(index)
index = ["Tom", "Bob", "Mary", "James", "Yafei"]
data = {
    "age": pd.Series([18, 30, 25, 40, 22], index=index),
    "city": pd.Series(["北京", "上海", "广州", "深圳", "晋城"],index=index)
}
user_info = pd.DataFrame(data)
user_info

,age,city
Tom,18,北京
Bob,30,上海
Mary,25,广州
James,40,深圳
Yafei,22,晋城


**当Series指定索引，DataFrame中也指定索引时会发生什么呢？**

In [2247]:
# 指定索引(index)
index = ["Tom", "Bob", "Mary", "James", "Yafei"]
data = {
    "age": pd.Series([18, 30, 25, 40, 22], index=index),
    "city": pd.Series(["北京", "上海", "广州", "深圳", "晋城"],index=index)
}
user_info = pd.DataFrame(data, index=["Tom","James", "Yafei"])
user_info

,age,city
Tom,18,北京
James,40,深圳
Yafei,22,晋城


**当每列的Series和DataFrame都指定索引且不相同时，DataFrame会根据传递的索引去每个Series中寻找在指定索引范围的值，相当于根据索引对每一列Series进行筛选操作。**  
请看下面这个示例，age列和city列为Series对象，但是没有指定index，这是index为默认值，即0-Series长度-1的数字索引，下例中的索引为0-4,。而创建DataFrame时指定的index为name列表，此时会去Series中寻找index属于name列表的元素添加到DataFrame中，发现age列和city列的Series的索引与name列表交集为空，故结果中name索引对应的age和city均为空值。

In [2248]:
data = {
    "age": pd.Series([18, 30, 25, 40, 22]),
    "city": pd.Series(["北京", "上海", "广州", "深圳", "晋城"])
}
user_info= pd.DataFrame(data, index=["Tom", "Bob", "Mary", "James", "Yafei"])
user_info

,age,city
Tom,NaN,NaN
Bob,NaN,NaN
Mary,NaN,NaN
James,NaN,NaN
Yafei,NaN,NaN


In [2249]:
data['age'].index

RangeIndex(start=0, stop=5, step=1)

In [2250]:
user_info.index

Index(['Tom', 'Bob', 'Mary', 'James', 'Yafei'], dtype='object')

**可以看到当列索引和DataFrame的索引同时指定时，若不一致的情况下会发生冲突，此时生成的DataFrame会以指定的index为准。**  
那么，我们如何将DataFrame的索引设置为name列呢？

In [2251]:
data = {
    "age": pd.Series([18, 30, 25, 40, 22]),
    "city": pd.Series(["北京", "上海", "广州", "深圳", "晋城"])
}
user_info = pd.DataFrame(data)
user_info

,age,city
0,18,北京
1,30,上海
2,25,广州
3,40,深圳
4,22,晋城


In [2252]:
user_info.index = ["Tom", "Bob", "Mary", "James", "Yafei"]
user_info

,age,city
Tom,18,北京
Bob,30,上海
Mary,25,广州
James,40,深圳
Yafei,22,晋城


上面说了索引的设置时候的注意点，那么columns也是一样的，当传入的data指定的列名与columns不一致时，生成的DataFrame的列会以指定的columns为准，并去data中去查找与columns存在交集的列，并将其放入到DataFrame中。也可以理解为起到了筛选data中列数据的作用。

In [2253]:
data = {
    "age": pd.Series([18, 30, 25, 40, 22]),
    "city": pd.Series(["北京", "上海", "广州", "深圳", "晋城"])
}
user_info = pd.DataFrame(data, columns=['city', 'hobby'])
user_info

,city,hobby
0,北京,NaN
1,上海,NaN
2,广州,NaN
3,深圳,NaN
4,晋城,NaN


### 方式2：字典套列表

In [2254]:
# 不指定index，index默认为数字，从0开始
data = {
    "name": ["Tom", "Bob", "Mary", "James", "Yafei"],
    "age": [18, 30, 25, 40, 22],
    "city": ["北京", "上海", "广州", "深圳", "晋城"]
}
user_info = pd.DataFrame(data)
user_info

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳
4,Yafei,22,晋城


In [2255]:
# 指定索引(index)
data = {
    "age": [18, 30, 25, 40, 22],
    "city": ["北京", "上海", "广州", "深圳", "晋城"]
}
index = ["Tom", "Bob", "Mary", "James", "Yafei"]
user_info = pd.DataFrame(data, index=index)
user_info

,age,city
Tom,18,北京
Bob,30,上海
Mary,25,广州
James,40,深圳
Yafei,22,晋城


### 方式3：列表套字典

In [2256]:
data = [
    {'name':'Tom','age':18,'city': "北京"},
    {'name':'Bob','age': 30, 'city': '上海'},
    {'name':'Mary','age': 25, 'city': '广州'},
    {'name':'James','age': 40, 'city': '深圳'}
]
user_info = pd.DataFrame(data)
user_info

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳


将 name 设置为 DataFrame 的索引号

In [2257]:
user_info.set_index(keys='name')

,age,city
name,,
Tom,18,北京
Bob,30,上海
Mary,25,广州
James,40,深圳


### 方式4：二维数组(列表套列表)

In [2258]:
data = [['Tom',18, "北京"],
        ['Bob', 30, "上海"],
        ['Mary', 25, "广州"],
        ['James', 40, "深圳"]]
columns = ['name', 'age', 'city']
user_info = pd.DataFrame(data=data, columns=columns)
user_info

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳


In [2259]:
user_info.set_index(keys='name')

,age,city
name,,
Tom,18,北京
Bob,30,上海
Mary,25,广州
James,40,深圳


In [2260]:
data = [[18, "北京"],
        [30, "上海"],
        [25, "广州"],
        [40, "深圳"]]
index = ['Tom', 'Bob', 'Mary', 'James']
columns = ['age', 'city']
user_info = pd.DataFrame(data=data, index=index, columns=columns)
user_info

,age,city
Tom,18,北京
Bob,30,上海
Mary,25,广州
James,40,深圳


### 方式5：from_dict
DataFrame from_dict（）方法用于将Dict转换为DataFrame对象。 此方法接受以下参数。
- data ：字典或类似数组的对象来创建DataFrame。
- orient ：数据的方向。 允许值为（“columns”，“index”），默认值为“columns”。
- columns ：当方向为“索引”时，用作DataFrame标签的值的列表。 如果与列方向一起使用，则会引发ValueError 。

当没有指定orient时，默认将key值作为列名。（列排列）

In [2261]:
data = {
    "name": ["Tom", "Bob", "Mary", "James", "Yafei"],
    "age": [18, 30, 25, 40, 22],
    "city": ["北京", "上海", "广州", "深圳", "晋城"]
}
user_info = pd.DataFrame.from_dict(data)
user_info

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳
4,Yafei,22,晋城


当指定orient=‘index’时，将key值作为行名。（行排列）

In [2262]:
user_info = pd.DataFrame.from_dict(data, orient='index')
user_info

,0,1,2,3,4
name,Tom,Bob,Mary,James,Yafei
age,18,30,25,40,22
city,北京,上海,广州,深圳,晋城


自定义列名

In [2263]:
user_info = pd.DataFrame.from_dict(data, orient='index', columns=['学生1', '学生2', '学生3', '学生4', '学生5'])
user_info

,学生1,学生2,学生3,学生4,学生5
name,Tom,Bob,Mary,James,Yafei
age,18,30,25,40,22
city,北京,上海,广州,深圳,晋城


我们可以发现，from_dict可以将字典类型的数据转换为DataFrame对象，而之前用DataFrame的构造函数也可以直接实现这一功能，但是，此方法没有选择使用基于索引的方向。

In [2264]:
data = {
    "name": ["Tom", "Bob", "Mary", "James", "Yafei"],
    "age": [18, 30, 25, 40, 22],
    "city": ["北京", "上海", "广州", "深圳", "晋城"]
}
user_info = pd.DataFrame(data)
user_info

,name,age,city
0,Tom,18,北京
1,Bob,30,上海
2,Mary,25,广州
3,James,40,深圳
4,Yafei,22,晋城


因此，当您想要索引方向时，请使用from_dict（）方法。 对于默认方案，最好使用DataFrame构造函数。

### 方式6：from_record
DataFrame.from_records()方法用于将结构化的数据或数组转换为DataFrame对象。 此方法接受以下参数：
- data: 结构化的数据数组，元组或dict序列，或DataFrame结构化的输入数据。
- index: 索引
- exclude : 序列，要排除的列或索引
- columns : 序列，列
- coerce_float : bool, default False，强制转换为浮点型
- nrows : int, default None,如果数据是迭代器，则要读取的行数。

In [2265]:
data = {
    "name": ["Tom", "Bob", "Mary", "James", "Yafei"],
    "age": [18, 30, 25, 40, 22],
    "city": ["北京", "上海", "广州", "深圳", "晋城"]
}
user_info = pd.DataFrame.from_records(data)
user_info

,age,city,name
0,18,北京,Tom
1,30,上海,Bob
2,25,广州,Mary
3,40,深圳,James
4,22,晋城,Yafei


In [2266]:
user_info = pd.DataFrame.from_records(data, exclude=['age'])
user_info

,city,name
0,北京,Tom
1,上海,Bob
2,广州,Mary
3,深圳,James
4,晋城,Yafei


In [2267]:
data = [[18, "北京"],
        [30, "上海"],
        [25, "广州"],
        [40, "深圳"]]
index = ['Tom', 'Bob', 'Mary', 'James']
user_info = pd.DataFrame.from_records(data, index=index, columns=['age', 'city'])
user_info

,age,city
Tom,18,北京
Bob,30,上海
Mary,25,广州
James,40,深圳


### 方式7：从文件中读取
数据分析过程中经常需要进行读写操作，Pandas实现了很多IO 操作的API，下面列举几个常见的常见的读取方法，其返回的数据类型均为DataFrame：
- read_csv: 读取csv文件
- read_excel: 读取excel文件
- read_html: 读取html文件
- read_sql: 读取sql数据
- read_json: 读取json数据

In [2268]:
df = pd.read_excel('data/scores.xlsx')
df.head()  # 查看前5行

,姓名,课程,学期,成绩
0,王大伟,大学英语,1,92
1,王大伟,大学英语,2,85
2,王大伟,大学英语,3,83
3,王大伟,大学英语,4,90
4,王大伟,高等数学,1,91


In [2269]:
df.tail()

,姓名,课程,学期,成绩
31,张明,高等数学,4,86
32,张明,大学体育,1,87
33,张明,大学体育,2,85
34,张明,大学体育,3,86
35,张明,大学体育,4,92


关于文件读取与写入相关知识，之后将会在文件IO专题部分讲解。

### 总结
关于DataFrame的创建，在pandas中为我们提供了多种方法，虽然方法较多，但是这也有另一个弊端，就是不知道选哪种，如何选的问题。我个人的使用经验来看，一般是**列表套字典**或**字典套列表**的方式采用DataFrame构造函数直接创建，当然，涉及文件读取的话，各种读取函数实际应用场景也相当多，这一部分就在之后介绍。

## 第3部分：DataFrame的属性
基本属性如下：
- df.shape   : 查看形状
- df.columns  : 查看列名
- df.index    : 查看索引
- df.dtype   : 查看列数据类型
- df.ndim    : 查看数据维度
- df.T      : 转置  
- df.values   : 将DataFrane转为数组类型

In [2270]:
# 指定索引(index)
data = {
    "age": [18, 30, 25, 40, 22],
    "city": ["北京", "上海", "广州", "深圳", "晋城"],
    'sex': ['男', '男', '女', '男', '男']
}
index = pd.Index(["Tom", "Bob", "Mary", "James", "Yafei"], name='name')  # pd.Index将列表转换为具有轴标签的索引列表
user_info = pd.DataFrame(data, index=index)
user_info

,age,city,sex
name,,,
Tom,18,北京,男
Bob,30,上海,男
Mary,25,广州,女
James,40,深圳,男
Yafei,22,晋城,男


In [2271]:
# 查看形状，即多少行多少列
user_info.shape

(5, 3)

In [2272]:
# 查看列名
user_info.columns

Index(['age', 'city', 'sex'], dtype='object')

In [2273]:
# 查看索引
user_info.index

Index(['Tom', 'Bob', 'Mary', 'James', 'Yafei'], dtype='object', name='name')

In [2274]:
# 查看各列数据类型
user_info.dtypes

age      int64
city    object
sex     object
dtype: object

In [2275]:
# 查看维度
user_info.ndim

2

In [2276]:
user_info.T

name,Tom,Bob,Mary,James,Yafei
age,18,30,25,40,22
city,北京,上海,广州,深圳,晋城
sex,男,男,女,男,男


In [2277]:
# 将DataFrame数据转为数组
user_info.values

array([[18, '北京', '男'],
       [30, '上海', '男'],
       [25, '广州', '女'],
       [40, '深圳', '男'],
       [22, '晋城', '男']], dtype=object)

## 第4部分：DataFrame的方法

### 基本方法
- df.head(n): 查看前n行数据，默认为5
- df.tail(n): 查看后n行数据，默认为5
- df.info(): 查看整体情况
- df.select_dtypes(include=['float64'])   # 选择特定类型的列  

In [2278]:
user_info.head(3)

,age,city,sex
name,,,
Tom,18,北京,男
Bob,30,上海,男
Mary,25,广州,女


In [2279]:
user_info.tail(3)

,age,city,sex
name,,,
Mary,25,广州,女
James,40,深圳,男
Yafei,22,晋城,男


In [2280]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, Tom to Yafei
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   age     5 non-null      int64 
 1   city    5 non-null      object
 2   sex     5 non-null      object
dtypes: int64(1), object(2)
memory usage: 332.0+ bytes


In [2281]:
user_info.select_dtypes(include='int64')

,age
name,
Tom,18
Bob,30
Mary,25
James,40
Yafei,22


### 描述与统计
整体描述与统计
>df.describe() #查看数字类型的列整体概况  
 df.describe(include=['object']) #查看非数字类型的列的整体情况  
 df.groupby(col1)[col2].count()      

单列描述与统计：通过df[col]选择该列数据，数据类型为Series，然后可以调用上一节介绍的Series的方法
> df[col].value_counts() #统计某列中每个值出现的次数，相当于分组   
 df[col].sum(): 对某列数据求和  
 ...  

In [2282]:
user_info.describe()

,age
count,5.000000
mean,27.000000
std,8.485281
min,18.000000
25%,22.000000
50%,25.000000
75%,30.000000
max,40.000000


In [2283]:
user_info.describe(include=['object'])

,city,sex
count,5,5
unique,5,2
top,北京,男
freq,1,4


In [2284]:
# 对年龄按照性别进行分组统计
user_info.groupby('sex')['age'].count()

sex
女    1
男    4
Name: age, dtype: int64

分组与统计我将会在之后的专题章节中进行讲解，这里只需要初步了解

In [2285]:
# 选择某列数据
sex = user_info['sex']
sex

d:\Program\Python39\lib\site-packages\IPython\core\displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


name
Tom      男
Bob      男
Mary     女
James    男
Yafei    男
Name: sex, dtype: object

对该列数据进行统计

In [2286]:
# 对sex进行频次统计
sex.value_counts()

sex
男    4
女    1
Name: count, dtype: int64

In [2287]:
# 对age列进行求和
user_info['age'].sum()

135

关于Series的其他统计方法请查找上一节内容

### 离散化
- cut(x,bins, right: bool = True,labels=None,
    retbins: bool = False,
    precision: int = 3,
    include_lowest: bool = False,
    duplicates: str = "raise",
    ordered: bool = True,
)  

- qcut(
    x,
    q,
    labels=None,
    retbins: bool = False,
    precision: int = 3,
    duplicates: str = "raise",
)

In [2288]:
pd.cut(user_info.age, 3)

name
Tom      (17.978, 25.333]
Bob      (25.333, 32.667]
Mary     (17.978, 25.333]
James      (32.667, 40.0]
Yafei    (17.978, 25.333]
Name: age, dtype: category
Categories (3, interval[float64, right]): [(17.978, 25.333] < (25.333, 32.667] < (32.667, 40.0]]

In [2289]:
pd.cut(user_info.age, [1, 18, 30, 50])

name
Tom       (1, 18]
Bob      (18, 30]
Mary     (18, 30]
James    (30, 50]
Yafei    (18, 30]
Name: age, dtype: category
Categories (3, interval[int64, right]): [(1, 18] < (18, 30] < (30, 50]]

In [2290]:
pd.cut(user_info.age, [1, 18, 30, 50], labels=["childhood", "youth", "middle"])

name
Tom      childhood
Bob          youth
Mary         youth
James       middle
Yafei        youth
Name: age, dtype: category
Categories (3, object): ['childhood' < 'youth' < 'middle']

cut 进行离散化之外，qcut 也可以实现离散化。cut 是根据每个值的大小来进行离散化的，qcut 是根据每个值出现的次数来进行离散化的。b

In [2291]:
pd.qcut(user_info.age, 3)

name
Tom      (17.999, 23.0]
Bob      (28.333, 40.0]
Mary     (23.0, 28.333]
James    (28.333, 40.0]
Yafei    (17.999, 23.0]
Name: age, dtype: category
Categories (3, interval[float64, right]): [(17.999, 23.0] < (23.0, 28.333] < (28.333, 40.0]]

### 排序
在进行数据分析时，少不了进行数据排序。Pandas 支持两种排序方式：按轴（索引或列）排序和按实际值排序。
- sort_index 方法是按照标签（行标签或列标签）进行排序
- sort_values 方法是按照某一列或多列的元素值进行排序

可能你已经发现了，这两个方法Series中也有，功能也极其相似，不同的是一个排序对象是Series，一个排序对象是DataFrame

In [2292]:
user_info.sort_index()

,age,city,sex
name,,,
Bob,30,上海,男
James,40,深圳,男
Mary,25,广州,女
Tom,18,北京,男
Yafei,22,晋城,男


In [2293]:
user_info.sort_index(axis=1, ascending=False) # 按照列进行倒序排

,sex,city,age
name,,,
Tom,男,北京,18
Bob,男,上海,30
Mary,女,广州,25
James,男,深圳,40
Yafei,男,晋城,22


In [2294]:
# 按照age列从小到大来排序
user_info.sort_values(by="age")

,age,city,sex
name,,,
Tom,18,北京,男
Yafei,22,晋城,男
Mary,25,广州,女
Bob,30,上海,男
James,40,深圳,男


In [2295]:
# 按照age和sex进行排序，优先排age，当age相同时，按sex排
user_info.sort_values(by=["age", "sex"])

,age,city,sex
name,,,
Tom,18,北京,男
Yafei,22,晋城,男
Mary,25,广州,女
Bob,30,上海,男
James,40,深圳,男


一般在排序后，我们可能需要获取最大的n个值或最小值的n个值，我们可以使用 nlargest和
nsmallest 方法来完成，这比先进行排序，再使用 head(n) 方法快得多。

In [2296]:
user_info['age'].nlargest(3)

name
James    40
Bob      30
Mary     25
Name: age, dtype: int64

In [2297]:
user_info['age'].nsmallest(3)

name
Tom      18
Yafei    22
Mary     25
Name: age, dtype: int64

### 类型操作
如果想要获取每种类型的列数的话，可以使用 df.dtypes 方法。  
- 如果想要转换数据类型的话，可以通过 astype 来完成。
- 有时候会涉及到将 object 类型转为其他类型，常见的有转为数字、日期、时间差，Pandas 中分别对应 to_numeric、to_datetime、to_timedelta 方法。

这里给这些用户都添加一些关于分数的信息，输入时以文本格式输入。现在将分数这一列转为数字，很明显，100分 并非数字，为了强制转换，我们可以传入 errors 参数，这个参数的作用是当强转失败时的处理方式。默认情况下，errors='raise'，这意味着强转失败后直接抛出异常，设置 errors='coerce' 可以在强转失败时将有问题的元素赋值为 pd.NaT（对于datetime和timedelta）或 np.nan（数字），设置 errors='ignore' 可以在强转失败时返回原有的数据。

In [2298]:
user_info.dtypes

age      int64
city    object
sex     object
dtype: object

In [2299]:
user_info.dtypes.value_counts()

object    2
int64     1
Name: count, dtype: int64

In [2300]:
user_info['age'].astype(float)

name
Tom      18.0
Bob      30.0
Mary     25.0
James    40.0
Yafei    22.0
Name: age, dtype: float64

In [2301]:
user_info["score"] = ["98", "78", "68", "88", "100分"]
user_info

,age,city,sex,score
name,,,,
Tom,18,北京,男,98
Bob,30,上海,男,78
Mary,25,广州,女,68
James,40,深圳,男,88
Yafei,22,晋城,男,100分


In [2302]:
user_info.dtypes

age       int64
city     object
sex      object
score    object
dtype: object

In [2303]:
# pd.to_numeric(user_info.score, errors="raise")  # 如果有值转换不成功会报错

# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# pandas\_libs\lib.pyx in pandas._libs.lib.maybe_convert_numeric()

# ValueError: Unable to parse string "100分"

# During handling of the above exception, another exception occurred:

# ValueError                                Traceback (most recent call last)
# <ipython-input-60-ff1e33582f2f> in <module>
# ----> 1 pd.to_numeric(user_info.score, errors="raise")

# F:\Anaconda3\lib\site-packages\pandas\core\tools\numeric.py in to_numeric(arg, errors, downcast)
#     150         coerce_numeric = errors not in ("ignore", "raise")
#     151         try:
# --> 152             values = lib.maybe_convert_numeric(
#     153                 values, set(), coerce_numeric=coerce_numeric
#     154             )

# pandas\_libs\lib.pyx in pandas._libs.lib.maybe_convert_numeric()

# ValueError: Unable to parse string "100分" at position 4

In [2304]:
pd.to_numeric(user_info.score, errors="coerce")

name
Tom      98.0
Bob      78.0
Mary     68.0
James    88.0
Yafei     NaN
Name: score, dtype: float64

In [2305]:
pd.to_numeric(user_info.score, errors="ignore")

name
Tom        98
Bob        78
Mary       68
James      88
Yafei    100分
Name: score, dtype: object

### 索引行函数

In [2306]:
user_info

,age,city,sex,score
name,,,,
Tom,18,北京,男,98
Bob,30,上海,男,78
Mary,25,广州,女,68
James,40,深圳,男,88
Yafei,22,晋城,男,100分


#### where
> where(cond,other=np.nan,inplace=False,axis=None,level=None,errors="raise",try_cast=False,)

默认不满足条件的行全部被设置为NaN

In [2307]:
user_info.where(user_info['age'] > 25)

,age,city,sex,score
name,,,,
Tom,NaN,NaN,NaN,NaN
Bob,30.0,上海,男,78
Mary,NaN,NaN,NaN,NaN
James,40.0,深圳,男,88
Yafei,NaN,NaN,NaN,NaN


通过这种方法筛选结果和[]操作符的结果完全一致：

In [2308]:
user_info.where(user_info['age'] > 25).dropna()

,age,city,sex,score
name,,,,
Bob,30.0,上海,男,78
James,40.0,深圳,男,88


In [2309]:
user_info[user_info['age'] > 25]

,age,city,sex,score
name,,,,
Bob,30,上海,男,78
James,40,深圳,男,88


第一个参数为布尔条件，第二个参数为填充值：

In [2310]:
user_info.where(user_info['age'] > 25, 25) # age小于25全部设置为25

,age,city,sex,score
name,,,,
Tom,25,25,25,25
Bob,30,上海,男,78
Mary,25,25,25,25
James,40,深圳,男,88
Yafei,25,25,25,25


#### mask
> mask( cond,other=np.nan,inplace=False,axis=None,level=None,errors="raise",try_cast=False)

mask函数与where功能上相反，其余完全一致，即对条件为True的单元进行填充。

In [2311]:
user_info.mask(user_info['age'] > 25) 

,age,city,sex,score
name,,,,
Tom,18.0,北京,男,98
Bob,NaN,NaN,NaN,NaN
Mary,25.0,广州,女,68
James,NaN,NaN,NaN,NaN
Yafei,22.0,晋城,男,100分


In [2312]:
user_info.mask(user_info['age'] > 25, 25) 

,age,city,sex,score
name,,,,
Tom,18,北京,男,98
Bob,25,25,25,25
Mary,25,广州,女,68
James,25,25,25,25
Yafei,22,晋城,男,100分


#### query
> query(self, expr, inplace=False, **kwargs)

query函数中的布尔表达式中，下面的符号都是合法的：行列索引名、字符串、and/not/or/&/|/~/not in/in/==/!=、四则运算符

In [2313]:
user_info.query('age > 20')

,age,city,sex,score
name,,,,
Bob,30,上海,男,78
Mary,25,广州,女,68
James,40,深圳,男,88
Yafei,22,晋城,男,100分


In [2314]:
user_info.query('(age > 20)&(sex == "男")')

,age,city,sex,score
name,,,,
Bob,30,上海,男,78
James,40,深圳,男,88
Yafei,22,晋城,男,100分


### 重复元素处理

#### duplicated方法
> duplicated(subset, keep='first')

- subset: 判断重复的列，默认为全部列,单列为字符串或列表类型，多列为列表类型
- keep: 重复的行标注为True的方式，first,last和False.False是所有重复元素均为True

In [2315]:
user_info.duplicated(subset='sex')

name
Tom      False
Bob       True
Mary     False
James     True
Yafei     True
dtype: bool

可选参数keep默认为first，即首次出现设为不重复，若为last，则最后一次设为不重复，若为False，则所有重复项为True

In [2316]:
user_info.duplicated(subset='sex', keep='last')

name
Tom       True
Bob       True
Mary     False
James     True
Yafei    False
dtype: bool

In [2317]:
user_info.duplicated(subset='sex', keep=False)

name
Tom       True
Bob       True
Mary     False
James     True
Yafei     True
dtype: bool

### drop_duplicates方法
> drop_duplicates(subset: ,keep="first",inplace=False,ignore_index=False) 

删除重复项，默认为只保留所有重复元素的第一个，其余都删除

In [2318]:
user_info.drop_duplicates(subset=['sex'])

,age,city,sex,score
name,,,,
Tom,18,北京,男,98
Mary,25,广州,女,68


保留重复元素的最后一个

In [2319]:
user_info.drop_duplicates(subset=['sex'], keep='last')

,age,city,sex,score
name,,,,
Mary,25,广州,女,68
Yafei,22,晋城,男,100分


在传入多列时等价于将多列共同视作一个多级索引，比较重复项：

In [2320]:
user_info.drop_duplicates(subset=['sex', 'age'])

,age,city,sex,score
name,,,,
Tom,18,北京,男,98
Bob,30,上海,男,78
Mary,25,广州,女,68
James,40,深圳,男,88
Yafei,22,晋城,男,100分


### 抽样函数

#### sample
> sample(n=None, frac=None,replace=False,weights=None,random_state=None,axis=None)

n为样本量

In [2321]:
user_info.sample(n=3)

,age,city,sex,score
name,,,,
Tom,18,北京,男,98
Yafei,22,晋城,男,100分
James,40,深圳,男,88


frac为抽样比

In [2322]:
user_info.sample(frac=0.5)

,age,city,sex,score
name,,,,
Mary,25,广州,女,68
James,40,深圳,男,88


replace为是否放回

In [2323]:
user_info.sample(n=5, replace=True)

,age,city,sex,score
name,,,,
James,40,深圳,男,88
James,40,深圳,男,88
Yafei,22,晋城,男,100分
Mary,25,广州,女,68
James,40,深圳,男,88


axis为抽样维度，默认为0，即抽行

In [2324]:
user_info.sample(n=5, axis=0)

,age,city,sex,score
name,,,,
James,40,深圳,男,88
Mary,25,广州,女,68
Tom,18,北京,男,98
Yafei,22,晋城,男,100分
Bob,30,上海,男,78


In [2325]:
user_info.sample(n=2, axis=1)

,age,sex
name,,
Tom,18,男
Bob,30,男
Mary,25,女
James,40,男
Yafei,22,男


weights为样本权重，自动归一化

In [2326]:
user_info.sample(n=3,weights=np.random.rand(user_info.shape[0]))

,age,city,sex,score
name,,,,
Mary,25,广州,女,68
Tom,18,北京,男,98
James,40,深圳,男,88


In [2327]:
# 以某一列为权重，这在抽样理论中很常见
# 抽到的概率与Math数值成正比
user_info.sample(n=3,weights=user_info['age'])

,age,city,sex,score
name,,,,
Tom,18,北京,男,98
Bob,30,上海,男,78
James,40,深圳,男,88


### 函数应用
虽说 Pandas 为我们提供了非常丰富的函数，有时候我们可能需要自己定制一些函数，并将它应用到 DataFrame 或 Series。常用到的函数有：map、apply、applymap。

- map 是 Series 中特有的方法，通过它可以对 Series 中的每个元素实现转换。如果我想通过年龄判断用户是否属于中年人（30岁以上为中年），通过 map 可以轻松搞定它。我想要通过城市来判断是南方还是北方，我可以这样操作。
- apply 方法既支持 Series，也支持 DataFrame，在对 Series 操作时会作用到每个值上，在对 DataFrame 操作时会作用到所有行或所有列（通过 axis 参数控制）。# 对 Series 来说，apply 方法 与 map 方法区别不大。
- applymap 方法针对于 DataFrame，它作用于 DataFrame 中的每个元素，它对 DataFrame 的效果类似于 apply 对 Series 的效果

In [2328]:
# 通过年龄判断用户是否属于中年人（30岁以上为中年）
user_info.age.map(lambda x: "yes" if x >= 30 else "no")

name
Tom       no
Bob      yes
Mary      no
James    yes
Yafei     no
Name: age, dtype: object

In [2329]:
# 通过城市来判断是南方还是北方
city_map = {
    "北京": "north",
    "上海": "south",
    "广州": "south",
    "深圳": "south",
    "晋城": "north",
}
user_info.city.map(city_map)

name
Tom      north
Bob      south
Mary     south
James    south
Yafei    north
Name: city, dtype: object

In [2330]:
# 求每一列的最大值
user_info.apply(lambda x: x.max(), axis=0)

age      40
city     深圳
sex       男
score    98
dtype: object

In [2331]:
# 求每一行的age加1
user_info.apply(lambda x: x.age + 1, axis=1)

name
Tom      19
Bob      31
Mary     26
James    41
Yafei    23
dtype: int64

In [2332]:
# 将每个值转化为小写字符串
user_info.applymap(lambda x: str(x) + 'applymap')

,age,city,sex,score
name,,,,
Tom,18applymap,北京applymap,男applymap,98applymap
Bob,30applymap,上海applymap,男applymap,78applymap
Mary,25applymap,广州applymap,女applymap,68applymap
James,40applymap,深圳applymap,男applymap,88applymap
Yafei,22applymap,晋城applymap,男applymap,100分applymap


## 第5部分：DataFrame的增删改查

### 数据准备

In [2333]:
# 指定索引(index)
data = {
    "age": [18, 30, 25, 40, 22],
    "city": ["北京", "上海", "广州", "深圳", "晋城"]
}
index = pd.Index(["Tom", "Bob", "Mary", "James", "Yafei"], name='name')  # pd.Index将列表转换为具有轴标签的索引列表
user_info = pd.DataFrame(data, index=index)
user_info

,age,city
name,,
Tom,18,北京
Bob,30,上海
Mary,25,广州
James,40,深圳
Yafei,22,晋城


DataFrame 就像带索引的 Series 字典，提取、设置、删除列的操作与字典类似：

### 索引/查找

**从查找方法角度分为三种：**
- 按标签/索引名/列名）查找
- 按序号/位置查找
- 按布尔条件查找  
  注：& | ~  且 或 非 多个布尔条件必须加小括号
- 按数据类型查找
   

**从语法角度分为5种：**
- df.col: 按列名选择列（不推荐）,若该列不存在会报错
- df.get(col, default): 按列名查找指定列数据，若不存在不错误，可以设置默认值
- df[]
   - [col]：按列名查找单列
   - [[col1. col2, ...]]: 按列名查找多列
   - [start: end]: 按序号进行切片查找
   - [bool表达式]：按布尔条件进行查找     

- df.loc[行标签，列标签]（推荐）
   - 行和列可以为行索引名或列名，表示查找单行或单列
   - 行和列可以为行索引列表或列名列表，表示查找多行或多列
   - 行和列可以为布尔表达式，根据布尔条件进行行和列的查找
- df.iloc[行序号，列序号]
   - 行和列可以为行序号或列序号，表示查找单行或单列
   - 行和列可以为行序号列表或列序号列表，表示查找多行或多列
   - 行和列可以为行序号切片或列序号切片，表示查找连续多行或连续多列
- df.select_dtypes(include=None, exclude=None): 按数据类型查找指定列

#### 查找指定列数据

**按列名查找**

按列名查找单列

In [2334]:
user_info['age']  # 单列

name
Tom      18
Bob      30
Mary     25
James    40
Yafei    22
Name: age, dtype: int64

In [2335]:
user_info.age

name
Tom      18
Bob      30
Mary     25
James    40
Yafei    22
Name: age, dtype: int64

In [2336]:
user_info.get('age')  # get方式获取指定列数据，若列不存在不会报错， 类似于字典的get

name
Tom      18
Bob      30
Mary     25
James    40
Yafei    22
Name: age, dtype: int64

In [2337]:
user_info.get('age2', '18')  # 可以设置默认值，若指定列不存在，则返回默认值

'18'

In [2338]:
user_info.loc[:, 'age']

name
Tom      18
Bob      30
Mary     25
James    40
Yafei    22
Name: age, dtype: int64

按列名查找多列

In [2339]:
user_info[['age', 'city']]  # 多列
# user_info.loc[:, ['age', 'city']]

,age,city
name,,
Tom,18,北京
Bob,30,上海
Mary,25,广州
James,40,深圳
Yafei,22,晋城


**按列序号查找**

In [2340]:
user_info.iloc[:, 1]

name
Tom      北京
Bob      上海
Mary     广州
James    深圳
Yafei    晋城
Name: city, dtype: object

In [2341]:
user_info.iloc[:, [0, 1]]

,age,city
name,,
Tom,18,北京
Bob,30,上海
Mary,25,广州
James,40,深圳
Yafei,22,晋城


按数据类型查找

In [2342]:
user_info.select_dtypes(include='int64')

,age
name,
Tom,18
Bob,30
Mary,25
James,40
Yafei,22


#### 查找指定行数据

根据行标签/索引

In [2343]:
user_info.loc['Tom']  # 单行

age     18
city    北京
Name: Tom, dtype: object

In [2344]:
user_info.loc[['Tom', 'Bob']]  # 多行

,age,city
name,,
Tom,18,北京
Bob,30,上海


根据行序号

In [2345]:
user_info.iloc[0]  # 第一行

age     18
city    北京
Name: Tom, dtype: object

In [2346]:
user_info.iloc[[0,1]]  # 前两行

,age,city
name,,
Tom,18,北京
Bob,30,上海


In [2347]:
user_info.iloc[0:2]  # 使用序号切片获取前两行

,age,city
name,,
Tom,18,北京
Bob,30,上海


In [2348]:
user_info[0:1] # 第0行 使用切片 

,age,city
name,,
Tom,18,北京


In [2349]:
user_info[0:2]  # 前两行  使用切片

,age,city
name,,
Tom,18,北京
Bob,30,上海


布尔查找

In [2350]:
# 查找年龄大于等于30的数据
user_info[user_info.age >= 30]

,age,city
name,,
Bob,30,上海
James,40,深圳


In [2351]:
# 查找年龄大于等于30且city为上海的数据
user_info[(user_info.age >= 30) & (user_info.city == '上海')]

,age,city
name,,
Bob,30,上海


In [2352]:
# 查找city为晋城或者深圳的数据
user_info[(user_info.city == '晋城') | (user_info.city == '深圳')]

,age,city
name,,
James,40,深圳
Yafei,22,晋城


还记得我们将Series时的isin方法吗？会对Series中每一个值进行成员运算，返回结果为布尔值。

In [2353]:
# 查找city为晋城或者深圳的数据
user_info[user_info.city.isin(['晋城','深圳'])]

,age,city
name,,
James,40,深圳
Yafei,22,晋城


In [2354]:
# 查找age不等于20，city不是北京的数据
user_info[(user_info.age != 20) & (user_info.city != '北京')]

,age,city
name,,
Bob,30,上海
Mary,25,广州
James,40,深圳
Yafei,22,晋城


In [2355]:
# 查找age不等于20，city不是北京的数据
user_info[~((user_info.age == 20) | (user_info.city == '北京'))]

,age,city
name,,
Bob,30,上海
Mary,25,广州
James,40,深圳
Yafei,22,晋城


#### 查找指定行指定列数据

按标签查找

按标签查找指定行指定列

In [2356]:
# 查找行标签为Tom的age列
user_info.loc['Tom', 'age']

18

按标签查找指定多行指定多列

In [2357]:
# 查找行标签为Tom和James的age和city列
user_info.loc[['Tom', 'James'], ['age', 'city']]

,age,city
name,,
Tom,18,北京
James,40,深圳


按布尔条件查找指定行指定列

In [2358]:
# 查找年龄大于等于30的人的city
user_info.loc[user_info.age >= 30, 'city']

name
Bob      上海
James    深圳
Name: city, dtype: object

按序号查找

按序号查询指定行指定列

In [2359]:
# 查找第1行第1列数据
user_info.iloc[0, 0]

18

按序号查找指定多行指定多列

In [2360]:
# 查找第1和第4行，第1和第2列数据
user_info.iloc[[0, 3], [0, 1]]

,age,city
name,,
Tom,18,北京
James,40,深圳


按序号切片查找指定多行指定多列

In [2361]:
# 查询前3行前2列数据
user_info.iloc[0:3, 0:2]
# user_info.iloc[:3, :2]  # 两种写法一样

,age,city
name,,
Tom,18,北京
Bob,30,上海
Mary,25,广州


### 添加
涉及到的相关方法如下：
- df[new_col] = data, data为类列表数据，添加列new_col
- df.append(other): other为DataFrame、Series或类dict,或此类型的list，在df末尾添加新行（Series和dict为单行，DataFrame或list是多行）
- df.insert(loc, column, value): 在制定位置插入新列
- df.loc[index， col] = data，添加行或新列(推荐)

In [2362]:
user_info

,age,city
name,,
Tom,18,北京
Bob,30,上海
Mary,25,广州
James,40,深圳
Yafei,22,晋城


#### 添加新列

In [2363]:
user_info['sex'] = ['男', '男', '女', '男', '男']
user_info

,age,city,sex
name,,,
Tom,18,北京,男
Bob,30,上海,男
Mary,25,广州,女
James,40,深圳,男
Yafei,22,晋城,男


在指定位置插入新列

In [2364]:
user_info.insert(1, 'hobby', ['羽毛球', '棒球', '舞蹈', '看球', '跑步'])
user_info

,age,hobby,city,sex
name,,,,
Tom,18,羽毛球,北京,男
Bob,30,棒球,上海,男
Mary,25,舞蹈,广州,女
James,40,看球,深圳,男
Yafei,22,跑步,晋城,男


DataFrame 提供了 assign() 方法，可以利用现有的列创建新列。

In [2365]:
user_info.assign(desc=user_info['city'] + '- '+ user_info['sex'] + '-' + user_info['hobby'])

,age,hobby,city,sex,desc
name,,,,,
Tom,18,羽毛球,北京,男,北京- 男-羽毛球
Bob,30,棒球,上海,男,上海- 男-棒球
Mary,25,舞蹈,广州,女,广州- 女-舞蹈
James,40,看球,深圳,男,深圳- 男-看球
Yafei,22,跑步,晋城,男,晋城- 男-跑步


#### 添加新行

最常用-推荐

In [2366]:
user_info.loc['Linda'] = [28, '乒乓球', '长治','女']
user_info

,age,hobby,city,sex
name,,,,
Tom,18,羽毛球,北京,男
Bob,30,棒球,上海,男
Mary,25,舞蹈,广州,女
James,40,看球,深圳,男
Yafei,22,跑步,晋城,男
Linda,28,乒乓球,长治,女


使用append方式在DataFrame末尾添加添加新行  
> append(other, ignore_index=False, verify_integrity=False, sort=False) 
>  - other: DataFrame、Series或类dict型对象，或这种对象的list
>  - ignore_index: bool,default False, 如果为True,新生成的DataFrame的行标签将会被标记为0,1,2...n-1
>  - verify_integrity : bool, default False, 如果为True，添加时会判断索引是否重复，如果之前添加过该索引数据，则raise ValueError
>  - sort : bool, default False，如果为True,如果两个对象的列没对齐，则进行排序

注意：append不会改变自身DataFrame，会将添加之后的数据生成一个新的DataFrame

In [2367]:
user1 = pd.Series(data=[23, '重庆', '火锅', '女'], index=['age', 'city', 'hobby', 'sex'], name='Lisa')
user_info.append(user1)

AttributeError: 'DataFrame' object has no attribute 'append'

插入字典对象，ignore_index必须为True

In [ ]:
user2 = {'age': 23, 'city': '重庆', 'hobby': '火锅', 'sex': '女'}
user_info.append(user2, ignore_index=True)

append也可以添加DataFrame对象，即实现两个DataFrame数据合并，关于这部分内容，我们将在之后合并与连接这一章节进行讲解

### 修改
涉及的相关方法如下：
- df.columns = columns: columns为列表， 设置新列名
- df.index = index, index为列表, 设置新索引
- df.rename(index=None, columns=None, inplace=False): index修改索引，columns修改列名，inplace是否修改原始数据
- df.loc[index， col] = value: 根据行或列标签修改指定数据
- df.iloc[iindex, icol] = value: 根据行位置和列位置修改指定数据

In [ ]:
user_info

#### 修改列名和索引

方式1：rename方式

In [ ]:
columns = {'age': '年龄', 'hobby': '爱好', 'city': '城市', 'sex': '性别'}
index = {'Tom': '汤姆', 'Bob': '鲍勃', 'Mary': '麦瑞', 'James': '詹姆斯', 'Yafei': '亚飞', 'Linda': '琳达'}
user_info.rename(columns=columns, index=index)
# 如果想修改原数据可以设置inplace=True
# user_info.rename(columns=columns, index=index, inplace=True)

In [ ]:
user_info

方式2：直接修改原数据

In [ ]:
user_info.columns = ['年龄', '爱好', '城市', '性别']
user_info.index = ['汤姆', '鲍勃', '麦瑞', '詹姆斯', '亚飞', '琳达']
user_info

#### 修改指定列数据
此列存在为修改，若不存在则为添加新列

In [ ]:
user_info['年龄'] = user_info['年龄'] + 1
user_info

#### 修改指定行数据

In [ ]:
user_info.loc['汤姆'] = [18, '排球', '天津', '男'] 
user_info

#### 修改指定行指定列

按标签修改

In [ ]:
# 修改Tom的城市为北京
user_info.loc['汤姆', '城市'] = '北京'
user_info

按序号位置修改

In [ ]:
# 这种方式使用的前提是了解指定位置的行或列的含义
# 修改第2行第2列的元素值为羽毛球
user_info.iloc[1, 1] = '羽毛球'
user_info

### 删除
- df.drop(index=None, columns=None, inplace=False)：index,根据索引删除行，columns，根据列名删除列，inplace是否修改原始数据，返回删除的行或列
- df.pop(col): 删除指定列，返回删除列
- del df[col]: 删除指定列，无返回值

#### 删除指定列

方式1：drop方式,可以选择是否修改原数据，有返回值

In [ ]:
# 删除城市列，不修改原数据
user_info.drop(columns=['城市'])

若修改原数据，可以设置inplace参数为True  
> user_info.drop(columns=['城市'], inplace=True) 

方式2：del方式，修改原数据，无返回值

In [ ]:
# 添加score列
import random
score = [random.randint(0, 100) for _ in range(user_info.shape[0])]
user_info['score'] = score
user_info

In [ ]:
del user_info['score']
user_info

方式3：pop，修改原数据，有返回值

In [ ]:
# 添加score列
user_info['score'] = score
user_info

In [ ]:
score_data = user_info.pop('score')
score_data

In [ ]:
user_info

#### 删除指定行

drop方式，可以设置是否修改原数据，有返回值

In [ ]:
user_info.drop(index=['汤姆'])

In [ ]:
user_info

## 第6部分：DataFrame的遍历

在实际数据处理的过程中，我们经常需要对某行或这某列的元素进行一些操作，这时就需要对元素进行遍历。那么pandas中如何对元素进行遍历呢?

![](https://img2018.cnblogs.com/blog/1476293/201903/1476293-20190312223939625-1790211193.png)

### 遍历列

In [ ]:
for col in user_info:
    print(col)

In [ ]:
for col in user_info:
    print(col)
    print(user_info[col])

In [ ]:
# 遍历单列
user_info['年龄'].apply(lambda x: x + 1)

In [ ]:
# 遍历所有列
def desc(row):
    print(row.name, row['汤姆'], row['鲍勃'], row['麦瑞'], row['詹姆斯'], row['亚飞'], row['琳达'])

user_info.apply(desc, axis=0)

### 遍历行

In [ ]:
def desc(row):
    print(row.name, row['年龄'], row['爱好'], row['城市'], row['性别'])

user_info.apply(desc, axis=1)

In [ ]:
for index, row in user_info.iterrows():
    print(index,row['年龄'], row['爱好'], row['城市'], row['性别'])

In [ ]:
for index in user_info.index:
    print(index, user_info.loc[index]['年龄'],user_info.loc[index]['爱好'], user_info.loc[index]['城市'],user_info.loc[index]['性别'])

In [ ]:
for row in user_info.values:
    print(row[0], row[1], row[2], row[3])

## 简单小结

本文介绍了pandas的第二大数据结构DataFrame的相关知识点。包含创建、属性、方法、以及增删改查和遍历。DataFrame是pandas数据的核心，所有的方法基本都是围绕Series和DataFrame展开的，本文介绍的当然不是全部，后面的章节会慢慢补充。当然，通过本文对于DataFrame的学习，相信你已经对DataFrame有了基本的了解，并掌握了其基本用法，别急，好戏才刚刚开始！